In [ ]:
import pandas as pd

# neo4j interface

In [ ]:
# KNOWLEDGE GRAPH CONSTRUCTION
import pandas as pd
import numpy as np

from neo4j import GraphDatabase

host = 'bolt://localhost:11003'
user = 'neo4j'
password = 'password'
driver = GraphDatabase.driver(host, auth=(user,password))

print("\nNeo4j credentials:\n host:",host,"\tuser:",user,"\tpassword:",password)

def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return [record.data() for record in result]


Neo4j credentials:
 host: bolt://localhost:11003 	user: neo4j 	password: password


# Helper functions

In [ ]:
def typeMapper(pandasTypes):
    '''
    Parameter:
        pandasType: list datatype in pandas.
    Returns:
        neoTypes: list of corresponding suitable neo4j datatypes parser when creating entities.
        ['toString, toInteger, datetime,...']
    '''
    typeMap = {
        'object':'toString',
        'datetime':'datetime',
        'M8': 'datetime',
        'int': 'toInteger',
        'float':'toFloat',
        'O':'toString',        
    }
    neoTypes = []
    for e in pandasTypes:
        for PType in typeMap:
            if PType.lower() in e.lower():
                neoTypes.append(typeMap[PType])
                break;
    return neoTypes

In [ ]:
def node(identifier, nodeType, attributes, values, attrTypes=None, typeParser=True):
    '''
    Parameters:
        identifier: Temporary unique name given to each node during creation; easier to refer it later(in the same command).
        nodeType: NodeLabel
        attributes: Node attribute names
        values: values for the corresponding node atttributes
        attrTypes: toFLoat, toInteger, toString, toBoolean; for corresponding attributes,
        typeParser: If True then adds toInteger(), toString(), etc to property values
            Keep True for creating nodes, False for matching nodes
    Returns:
        (identifier:nodeType {attribute: attrType(values)...}) - a node in CYPHER query
    NULL/nan/.. properties are not considered.
    **Creates copy of dataframe.**
    '''
    if attrTypes is None:
        attrTypes = [None]*len(attributes)
    cmd = '(' + str(identifier) + ':' + str(nodeType) + '  { '
    for attr, val, typ in zip(attributes, values, attrTypes):
        if not pd.isnull(val):
            cmd += ' , ' + str(attr) + ':'
            if typeParser and typ:
                cmd += str(typ)
            cmd +='('
            if typ in ['toString','datetime','quote']:
                cmd += '"'
            cmd += str(val)
            if typ in ['toString','datetime','quote']:
                cmd += '"'
            cmd +=') '
    cmd += ' })'
    return cmd

In [ ]:
import re

def createNodes(df, cols, nodeLabel, unique=False, execute=False, cmds=True):
    '''
    Parameters:
        df: dataframe,
        cols: list of cols that are attributes of this node label,
        <str> nodeLabel: NodeLabel,
        <bool> unique: default: False; If true then df[cols].drop_duplicates(),
        <bool> execute: If True, then cypher queries run on neo4j database,
        <bool> cmds: If True then returns list of commands.
    Returns:
        If cmds=True then returns list of cypher commands else None.
    '''
    tmp = df[cols].copy()
    if unique:
        tmp = tmp.drop_duplicates(ignore_index=True)

    #column types in pandas; attribute type in neo4j
    colTypes = [ dtyp.name for dtyp in tmp.dtypes.tolist()]
    neoTypes = typeMapper(colTypes)
    #if there is datetime column; extract just the date
    for i in range(len(cols)):
        if neoTypes[i]=='datetime':
            tmp[cols[i]] = pd.to_datetime(df[cols[i]]).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
   
    cypher_cmds = []
    for idx, row in tmp.iterrows():
        cmd  = 'CREATE ' + node(
            identifier = '',
            nodeType = nodeLabel,
            attributes= [re.sub(r'[^a-zA-Z0-9]', '_', col).lower() for col in cols],
            values = [row[col] for col in cols], 
            attrTypes = neoTypes
        )
        if execute:
            neo4j_query(cmd)
        if cmds:
            cypher_cmds.append(cmd)
    return cypher_cmds if cmds else None

In [ ]:
def __relsValidate(nodes, rels):
    '''Checks if all the end node types of rels actually exists in nodes or not.'''
    for rel, endVs in rels.items():
        for endV in endVs:
            if endV not in nodes:
                raise TypeError(f'"{endV}" NodeLabel is not present in given nodes, for relationship "{rel}".')
def createRelations(df, nodes, rels, execute=False, cmds=True):
    '''
    Parameters:
        df: pandas dataframe,
        nodes: a dict of format: { 'NodeLabel':[list of primary key columns for this nodeLabel in df],...},
        rels: a dict of format: {'RelationName': [sourceNodeLabel, destNodeLabel], ...}
        execute: <boolean> default False; if true then commands are executed on neo4j database,
        cmds: <boolean> default True; returns list of create relationship CYPHER commands.
    Returns:
        returns list of create relationship CYPHER commands if 'cmds' is True else None.
    '''
    __relsValidate(nodes, rels)
    tmp = df
    cypher_cmds=[]
    for idx, row in tmp.iterrows(): #for each row of df
        cmd = 'MATCH '
        # Match all node types
        for nodeLabel, keys in nodes.items():
            cmd += '\n\t' + node(
                    identifier = nodeLabel+str(idx),
                    nodeType = nodeLabel,
                    attributes= [re.sub(r'[^a-zA-Z0-9]', '_', col).lower() for col in keys],
                    values = [row[col] for col in keys], 
                    attrTypes = ['quote']*len(keys),
                    typeParser = False
            )+","
        cmd = cmd[:-1] #remove last comma
        #set all relations
        for rel, ends in rels.items():
            cmd += f'\nCREATE ({ends[0]+str(idx)})-[:{rel.upper()}]->({ends[1]+str(idx)})'
        if execute:
            neo4j_query(cmd)
        if cmds:
            cypher_cmds.append(cmd)
    return cypher_cmds if cmds else None

# Load the data
with which neo4j will be populated

In [ ]:
import pandas as pd

df = pd.read_csv('./csvs/input_to_neo4j.csv')
print(f'Shape : {df.shape}')
df.head(4)

## Change Dataypes

In [ ]:
import warnings

def changeDTypes(df, cols, toTypes):
    #TODO: both 'int' and 'float' are converted to just numericals; have to work with NaN.
    for col, totype in zip(cols, toTypes):
        if col in df.columns:
            if totype == 'int':
                df[col] = pd.to_numeric(df[col], errors='coerce')
            elif totype == 'float':
                df[col] = pd.to_numeric(df[col], errors='coerce')
            elif totype == 'datetime':
                df[col] = pd.to_datetime(df[col], dayfirst=True) #TODO: without time
            elif totype == 'string':
                df[col] = df[col].astype(str)
        else:
            warnings.warn( f'"{col}" column not present in dataframe.')
    return df

In [ ]:
cols = ['Claim No','Card No','Diagnosis','Surgery/Therapy', 'Provider Name','Date of Admit', 'Status', 'Status Date', 'Est Amt','Pre Auth Appr Amt', 'Surgery/Therapy Date', 'Discharged']
dtypes = ['str', 'str', 'str', 'str', 'str', 'datetime', 'str','datetime', 'float', 'float', 'datetime', 'datetime']
df = changeDTypes(df, cols, dtypes)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863 entries, 0 to 862
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Claim No              863 non-null    object        
 1   Card No               863 non-null    object        
 2   Diagnosis             863 non-null    object        
 3   Surgery/Therapy       863 non-null    object        
 4   Provider Name         863 non-null    object        
 5   Date of Admit         863 non-null    datetime64[ns]
 6   Status                863 non-null    object        
 7   Status Date           863 non-null    datetime64[ns]
 8   Est Amt               853 non-null    float64       
 9   Pre Auth Appr Amt     853 non-null    float64       
 10  Surgery/Therapy Date  863 non-null    datetime64[ns]
 11  Discharged            832 non-null    datetime64[ns]
 12  Length of Stay        832 non-null    float64       
 13  Treatment Duration  

# Create Patients

In [ ]:
%%time

# cols of patient
cols = ['Card No']

tmp = df[cols]
tmp['Name'] = [f'Patient{i}' for i in range(len(df))]

cmds = createNodes(
    df = tmp,
    cols = ['Name'] + cols,
    nodeLabel= 'Patient',
    unique = True,
    execute= False,
    cmds =True
)
cmds[:3]
# print('\n'.join(cmds))

# Create Claims

In [ ]:
%%time

cols = ['Claim No', 'Status', 'Status Date', 'Est Amt', 'Pre Auth Appr Amt', 'Claim Duration']

cmds = createNodes(
    df = df,
    cols = cols,
    nodeLabel= 'Claim',
    unique = True,
    execute= False,
    cmds =True
)
cmds[:3]
# print('\n'.join(cmds))

# Create EHCP

In [ ]:
%%time

cols = ['Provider Name']

cmds = createNodes(
    df = df,
    cols = cols,
    nodeLabel= 'EHCP',
    unique = True,
    execute= False,
    cmds =True
)
cmds[:3]
# print('\n'.join(cmds))

# Create Treatment

In [ ]:
%%time

cols = ['Claim No', 'Diagnosis', 'Surgery/Therapy', 'Date of Admit', 'Surgery/Therapy Date', 'Discharged',
       'Length of Stay', 'Treatment Duration']

cmds = createNodes(
    df = df,
    cols = cols,
    nodeLabel= 'Treatment',
    unique = False,
    execute= False,
    cmds =True
)
cmds[:3]
# print('\n'.join(cmds))

# Create relationships
for each row

In [ ]:
%%time

nodes = {
    'Patient':['Card No'],
    'EHCP':['Provider Name'],
    'Treatment':['Claim No'],
    'Claim':['Claim No'],
    'Surgery_Therapy':['Surgery/Therapy']
}
rels = {
    'TREATED_BY': ['Patient','EHCP'],
    'PROVIDED':['EHCP','Treatment'],
    'TREATED_BY':['Patient','EHCP'],
    'APPLIES':['EHCP','Claim'],
    'FOR':['Claim','Treatment'],
    'BENEFICIARY_TO':['Patient','Claim'],
    
}

cmds = createRelations(
    df,
    nodes,
    rels,
    execute=False,
    cmds=True
)
[print(x) for x in cmds[:4]]
# print('\n'.join(cmds))

In [ ]:
def createRelations(df, nodes, rels):
    '''
    Parameters:
        df: pandas dataframe,
        nodes: a dict of format: { 'NodeLabel':[list of primary key columns for this nodeLabel in df],...},
        rels: a dict of format: {'RelationName': [sourceNodeLabel, destNodeLabel], ...}
        execute: <boolean> default False; if true then commands are executed on neo4j database,
        cmds: <boolean> default True; returns list of create relationship CYPHER commands.
    Returns:
        returns list of create relationship CYPHER commands if 'cmds' is True else None.
    '''
    tmp = df
    cypher_cmds=[]
    for idx, row in tmp.iterrows(): #for each row of df
        cmd = 'MATCH '
        # Match all node types
        for nodeLabel, keys in nodes.items():
            cmd += '\n\t' + node(
                    identifier = nodeLabel+str(idx),
                    nodeType = nodeLabel,
                    attributes= [re.sub(r'[^a-zA-Z0-9]', '_', col).lower() for col in keys],
                    values = [row[col] for col in keys], 
                    attrTypes = ['quote']*len(keys),
                    typeParser = False
            )+","
        cmd = cmd[:-1] #remove last comma
        #set all relations
        exe_cmds = '' #commands to execute
        ret_cmds = '' #commands to return
        for rel, specs in rels.items():
            rel_cmd = f'\nCREATE ({specs["ends"][0]+str(idx)})-[:{rel.upper()}]->({specs["ends"][1]+str(idx)})'
            if specs["execute"]:
                exe_cmds += rel_cmd
            if specs["cmds"]:
                ret_cmds += rel_cmd
        cypher_cmds.append(cmd + ret_cmds)
        if exe_cmds != '':
            neo4j_query(cmd + exe_cmds)
    return cypher_cmds

def createNodesAndRelations(df,nodes=None, rels=None):
    node_cmds = []
    rel_cmds = []
    if nodes is not None:
        for nodeLabel, specs in nodes.items():
            cmds = createNodes(
                df = df,
                cols = specs["cols"],
                nodeLabel= nodeLabel,
                unique = specs["unique"],
                execute= specs["execute"],
                cmds = specs["cmds"]
            )
            node_cmds.append(cmds)
    if rels is not None:
        nodes_used=[]
        for rel, specs in rels.items():
            nodes_used.extend(specs['ends'])
        nodes_used = set(nodes_used)
        rel_nodes = {}
        for x in nodes_used:
            rel_nodes[x] = nodes[x]['primary_key']
        rel_nodes
        rel_cmds = createRelations(df, rel_nodes, rels)
    return node_cmds, rel_cmds

In [ ]:
%%time

execute = False
nodes = {
    'Patient':{
        "cols":['Card No'],
        "primary_key":['Card No'],
        "unique":True,
        "execute": execute,
        "cmds":True
    },
    'EHCP':{
        "cols": ['Provider Name'],
        "primary_key":['Provider Name'],
        "unique":True,
        "execute": execute,
        "cmds":True
    },
    'Treatment':{
        "cols": ['Claim No', 'Diagnosis', 'Date of Admit', 'Surgery/Therapy Date', 'Discharged',
       'Length of Stay', 'Treatment Duration'],
        "primary_key":['Claim No'],
        "unique":False,
        "execute": execute,
        "cmds":True
    },
    'Claim':{
        "cols":['Claim No', 'Status', 'Status Date', 'Est Amt', 'Pre Auth Appr Amt', 'Claim Duration'],
        "primary_key":['Claim No'],
        "unique":True,
        "execute": execute,
        "cmds":True
    },
    'Surgery_Therapy':{
        "cols": ['Surgery/Therapy'],
        "primary_key":['Surgery/Therapy'],
        "unique":True,
        "execute": execute,
        "cmds":True
    },
}
execute = True
rels = {
    'TREATED_BY': {
        "ends": ['Patient','EHCP'],
        "execute": execute,
        "cmds":True
    },
    'PROVIDED': {
        "ends": ['EHCP','Treatment'],
        "execute": execute,
        "cmds":True
    },
    'APPLIES': {
        "ends": ['EHCP','Claim'],
        "execute": execute,
        "cmds":True
    },
    'APPLIED_FOR': {
        "ends": ['Claim','Treatment'],
        "execute": execute,
        "cmds":True
    },
    'BENEFICIARY_TO': {
        "ends": ['Patient','Claim'],
        "execute": execute,
        "cmds":True
    },
    'HAS':{
        "ends": ['Treatment','Surgery_Therapy'],
        "execute": execute,
        "cmds":True
    },
    'SPECIALIZES_IN':{
        "ends": ['EHCP','Surgery_Therapy'],
        "execute": execute,
        "cmds":True
    }
    
}

x = createNodesAndRelations(df=df,nodes= nodes, rels= rels)
# print('\n'.join(x[0][2]))


CPU times: user 1.18 s, sys: 43.6 ms, total: 1.23 s
Wall time: 39.6 s


In [ ]:
print('\n'.join(x[1]))